In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import time
import os

In [8]:
spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config(
        "spark.driver.extraClassPath",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config("spark.executor.cores", '8')
    .config("spark.sql.compute.ops_on_diff_frames", True) 
    .getOrCreate())

24/04/29 16:51:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
import pyspark.pandas as ps

con="jdbc:sqlite:{}/database.sqlite".format(os.getcwd())

/home/marinevas/.asdf/installs/python/3.10.13/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [20]:
rankings = ps.read_sql("rankings", con=con)
# rankings = rankings.rename(columns={'player': 'player_id'})
rankings.head(2)

,ranking_date,rank,player,points
0,20000110,1,101736,4135.0
1,20000110,2,102338,2915.0


In [21]:
players = ps.read_sql("players", con=con)
players.head(2)

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581


In [34]:
matches = ps.read_sql('matches', con=con)
# matches = matches.rename(columns={'winner_id': 'player_id'})
# matches.dropna()
# matches['winner_id'] = matches['winner_id'].astype(int)
matches.head(2)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner1_id,winner2_id,loser1_id,loser2_id,winner1_name,winner1_hand,winner1_ht,winner1_ioc,winner1_age,winner2_name,winner2_hand,winner2_ht,winner2_ioc,winner2_age,loser1_name,loser1_hand,loser1_ht,loser1_ioc,loser1_age,loser2_name,loser2_hand,loser2_ht,loser2_ioc,loser2_age,winner1_rank,winner1_rank_points,winner2_rank,winner2_rank_points,loser1_rank,loser1_rank_points,loser2_rank,loser2_rank_points
0,2017-M020,Brisbane,Hard,32,A,20170102.0,271,104678.0,None,None,Viktor Troicki,R,193.0,SRB,30.8,106415.0,None,Q,Yoshihito Nishioka,L,170.0,JPN,21.2,6-4 7-5,3,R32,91.0,11.0,5.0,64.0,45.0,35.0,6.0,11.0,1.0,3.0,0.0,1.0,82.0,53.0,33.0,13.0,11.0,6.0,10.0,29.0,1385.0,100.0,604.0,NaN,None,None,NaN,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-M020,Brisbane,Hard,32,A,20170102.0,272,106378.0,None,None,Kyle Edmund,R,188.0,GBR,21.9,124014.0,None,Q,Ernesto Escobedo,R,185.0,USA,20.4,7-6(4) 7-6(6),3,R32,130.0,11.0,2.0,83.0,48.0,37.0,19.0,12.0,2.0,3.0,11.0,3.0,113.0,67.0,39.0,27.0,12.0,9.0,10.0,45.0,1001.0,141.0,443.0,NaN,None,None,NaN,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
rankings = rankings.to_spark()
players = players.to_spark()
matches = matches.to_spark()

/home/marinevas/.asdf/installs/python/3.10.13/lib/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


count the number of matches and average rank of players with rankings above 100

o tempo deu sempre o mesmo para 2, 4 e 8 cores...

In [11]:
query = '''
SELECT players.name_first, players.name_last, 
       AVG(rankings.rank) AS avg_rank, 
       COUNT(matches.match_num) AS total_matches
FROM players
JOIN rankings ON players.player_id = rankings.player
JOIN matches ON players.player_id = matches.winner_id
WHERE rankings.rank <= 100
GROUP BY players.name_first, players.name_last
ORDER BY avg_rank DESC
'''

In [ ]:
query = '''
SELECT players.name_first, players.name_last, 
       AVG(rankings.rank) AS avg_rank, 
       COUNT(matches.match_num) AS total_matches
FROM players
JOIN rankings ON players.player_id = rankings.player
JOIN matches ON players.player_id = matches.winner_id
GROUP BY players.name_first, players.name_last
ORDER BY avg_rank DESC
'''

In [ ]:
query = '''
SELECT ranking_date, AVG(rank) AS avg_rank, SUM(points) AS total_points
FROM rankings_1
GROUP BY ranking_date
UNION
SELECT ranking_date, AVG(rank) AS avg_rank, SUM(points) AS total_points
FROM rankings_2
GROUP BY ranking_date;
'''

In [12]:
start_time = time.time()
df = ps.read_sql_query(query, con=con)
print(time.time() - start_time)
# df.head(2)

,name_first,name_last,avg_rank,total_matches
0,Alex,Lopez Moron,100.000000000000000000,536.000000000000000000
1,Francisco,Montana,100.000000000000000000,91.000000000000000000


# query 3M spark

In [ ]:
# from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Rankings Aggregation") \
    .config(
        "spark.driver.extraClassPath",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd())) \
    .config("spark.executor.cores", '1') \
    .getOrCreate()

start_time = time.time()
# Load the rankings data into DataFrames (replace "path_to_rankings_1" and "path_to_rankings_2" with actual file paths)
url = "jdbc:sqlite:{}/database.sqlite".format(os.getcwd())

rankings_df1 = spark.read.jdbc(url=url, table="rankings")
rankings_df2 = spark.read.jdbc(url=url, table="rankings")

# Apply aggregation functions
aggregated_df1 = rankings_df1.groupBy("ranking_date") \
    .agg(avg("rank").alias("avg_rank"), sum("points").alias("total_points"))

aggregated_df2 = rankings_df2.groupBy("ranking_date") \
    .agg(avg("rank").alias("avg_rank"), sum("points").alias("total_points"))

# Union the two DataFrames
result_df = aggregated_df1.union(aggregated_df2)

print(time.time() - start_time)

In [3]:
from pyspark.sql.functions import col, avg, count

spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config(
        "spark.driver.extraClassPath",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config("spark.executor.cores", '1')
    .config("spark.sql.compute.ops_on_diff_frames", True) 
    .getOrCreate())

total_time = []
for i in range(1000):
    start_time = time.time()
    url = "jdbc:sqlite:{}/database.sqlite".format(os.getcwd())
    
    players_df = spark.read.jdbc(url=url, table="players")
    rankings_df = spark.read.jdbc(url=url, table="rankings")
    matches_df = spark.read.jdbc(url=url, table="matches")
    
    # Perform joins and filters
    merged_df = players_df.join(rankings_df, players_df["player_id"] == rankings_df["player"]) \
        .join(matches_df, players_df["player_id"] == matches_df["winner_id"]) \
        .filter(rankings_df["rank"] <= 100)
    
    # Perform aggregation and ordering
    result_df = merged_df.groupBy("name_first", "name_last") \
        .agg(avg("rank").alias("avg_rank"), count("match_num").alias("total_matches")) \
        .orderBy(col("avg_rank").desc())
    
    total_time.append(time.time() - start_time)

print(sum(total_time) / len(total_time))
# # Show the result
# result_df.show()

# spark.stop()

24/04/29 16:22:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+------------+-----------------+-------------+
|name_first|   name_last|         avg_rank|total_matches|
+----------+------------+-----------------+-------------+
|      Jeff| Salzenstein|            100.0|          212|
|     Terry|     Addison|            100.0|           71|
| Francisco|     Montana|            100.0|           91|
|     Terry|        Ryan|            100.0|           52|
|      Alex| Lopez Moron|            100.0|          536|
|      Paul|      Wekesa|            100.0|          393|
|Juan Pablo|      Guzman|            100.0|          828|
|     Jurij|    Rodionov|            100.0|          454|
|     Melle|Van Gemerden|            100.0|          396|
|  Vincenzo|  Santopadre|            100.0|          313|
|      Hugo|     Armando|            100.0|          290|
|John Chris|       Lewis|            100.0|           20|
|      Tony|      Graham|            100.0|           42|
|      Luca|       Vanni|            100.0|          912|
|    Emilio|  

# query 6M sql

In [ ]:
query = '''
SELECT r1.ranking_date AS date1,
       r1.rank AS rank1,
       r1.points AS points1,
       r2.ranking_date AS date2,
       r2.rank AS rank2,
       r2.points AS points2
FROM rankings r1
JOIN rankings r2 ON r1.ranking_date < r2.ranking_date
'''

spark.stop()

spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config(
        "spark.driver.extraClassPath",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config("spark.executor.cores", '1')
    .config("spark.sql.compute.ops_on_diff_frames", True) 
    .getOrCreate())

import pyspark.pandas as ps

con="jdbc:sqlite:{}/database.sqlite".format(os.getcwd())

start_time = time.time()
df = ps.read_sql_query(query, con=con)
print(time.time() - start_time)

# query 6M spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config(
        "spark.driver.extraClassPath",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config("spark.executor.cores", '8')
    .config("spark.sql.compute.ops_on_diff_frames", True) 
    .getOrCreate())

start_time = time.time()

# Load the rankings data into a DataFrame (replace "path_to_rankings" with the actual file path)
url = "jdbc:sqlite:{}/database.sqlite".format(os.getcwd())
rankings_df = spark.read.jdbc(url=url, table="rankings")

rankings_df.repartition(7)

# Self-join the rankings DataFrame
rankings_df = rankings_df.alias("r1") \
    .join(rankings_df.alias("r2"), col("r1.ranking_date") < col("r2.ranking_date")) \
    .select(col("r1.ranking_date").alias("date1"),
            col("r1.rank").alias("rank1"),
            col("r1.points").alias("points1"),
            col("r2.ranking_date").alias("date2"),
            col("r2.rank").alias("rank2"),
            col("r2.points").alias("points2"))

print(time.time() - start_time)

spark.stop() 

In [79]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, count

# Initialize SparkContext
# sc = SparkContext("local", "SQLite RDD Example")

# Initialize SparkSession
# spark = SparkSession(sc)

sc = spark.sparkContext

# Define the file paths
players_path = "file:///{}/database.sqlite".format(os.getcwd())
rankings_path = "file:///{}/database.sqlite".format(os.getcwd())
matches_path = "file:///{}/database.sqlite".format(os.getcwd())

# Load data into RDDs
players_rdd = sc.textFile(players_path)
rankings_rdd = sc.textFile(rankings_path)
matches_rdd = sc.textFile(matches_path)

# Transform RDDs into key-value pairs
players_kv = players_rdd.map(lambda line: line.split(",")).map(lambda fields: (fields[0], (fields[1], fields[2])))
rankings_kv = rankings_rdd.map(lambda line: line.split(",")).map(lambda fields: (fields[0], fields[1]))
matches_kv = matches_rdd.map(lambda line: line.split(",")).map(lambda fields: (fields[0], fields[1]))

# Perform the joins and filters
merged_rdd = players_kv.join(rankings_kv).map(lambda x: (x[0], (x[1][0][0], x[1][0][1], x[1][1])))
merged_rdd = merged_rdd.join(matches_kv).map(lambda x: (x[1][0][0], x[1][0][1], x[1][0][2], x[1][1]))

# Filter based on rankings rank
filtered_rdd = merged_rdd.filter(lambda x: int(x[2]) <= 100)

# Perform aggregation
aggregated_rdd = filtered_rdd.map(lambda x: ((x[0], x[1]), (int(x[2]), 1))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .mapValues(lambda x: (x[0] / x[1], x[1]))

# Convert RDD to DataFrame for sorting
# result_df = aggregated_rdd.map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][1])).toDF(["name_first", "name_last", "avg_rank", "total_matches"])

# # Order by avg_rank in descending order
# result_df = result_df.orderBy(col("avg_rank").desc())

# Show the result
aggregated_rdd.take(10)

# Stop SparkSession
spark.stop()


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand